In [39]:
!pip install scikit-surprise

In [40]:
pip install pickle-mixin

In [41]:
import os
import sys
import pickle

filenamelst = ['a.txt','b.py','c.ipynb']
filenamelst_abspathname = os.path.abspath('filenamelst.pickle')
print(filenamelst_abspathname)
picklefile = open(str(filenamelst_abspathname),'wb')
pickle.dump(filenamelst, picklefile)
picklefile.close()

/content/filenamelst.pickle


In [42]:
from flask import Flask, request, render_template
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.stem.snowball import SnowballStemmer
import pandas as pd
import numpy as np
import pickle
import sys
from surprise import Dataset
from surprise import Reader
from surprise import SVD

#pickle part 


def stem_and_vectorize_products_based_on_metadata(product_input):

    word_list = nltk.word_tokenize(product_input)
    input_stemmed = ' '.join([stemmer.stem(word) for word in word_list])
    vec = new_stem_count_vec.transform(np.array(input_stemmed).reshape(1,))
    
    simil = cosine_similarity(vec, new_stem_count_vec_matrix)
    simil_shape = simil.reshape(new_stem_count_vec_matrix.shape[0],)
    simil_scores = pd.DataFrame(data=simil_shape, index=prod_index, columns=['score'])

    # Don't return scores of zero or below
    non_zero_scores = simil_scores[simil_scores['score'] > 0]
    
    if len(non_zero_scores) == 0:
        return 0, 'None'
    
    if len(non_zero_scores) < 10:
        item_count = len(non_zero_scores)
    else:
        item_count = 10
    
    similarity_scores = simil_scores.sort_values(['score'], ascending=False)[:item_count]

    return item_count, (products_desc.iloc[list(similarity_scores.index)]).to_html(index=False, justify='center', classes='table1', border=2)

def get_sample_product(aisle=None):
    if aisle:
        product = products_desc[products_desc['Aisle'].str.contains(aisle)].sample(1)
    else:
        product = products_desc.sample(1)
    name = product['Product Name'].iloc[0]
    prod_aisle = product['Aisle'].iloc[0]
    prod_id = product['Product ID'].iloc[0]  
    return name, prod_aisle, prod_id

def generate_recs(ratings_list, n_to_rec, percent_diverse, rec_aisle=None):
    # Converting ratings to user ratings
    userID = 300000
    user_rating =[]
    for product, rating in ratings_list:
        rating_one_product = {'user_id':userID,'product_id':product,'rating':rating}
        user_rating.append(rating_one_product) 

    # adding new ratings generated to Dataframe
    print('Creating ratings dataset...')
    new_ratings_df = new_rec_df.append(user_rating, ignore_index=True)
    new_data = Dataset.load_from_df(new_ratings_df, reader)
    
    # train a model with new DataFrame
    print('Training recommendation model...')
    new_user_svd = SVD(n_factors = 20, n_epochs = 10, lr_all = 0.005, reg_all = 0.4)
    new_user_svd.fit(new_data.build_full_trainset())
    
    # Predictions
    print('Making predictions...')
    list_of_products = []
    for product in new_ratings_df['product_id'].unique():
        product_name = products_desc[products_desc['Product ID'] == product]['Product Name'].iloc[0]
        product_aisle = products_desc[products_desc['Product ID'] == product]['Aisle'].iloc[0]
        list_of_products.append((product, round(new_user_svd.predict(300000, product)[3], 3), product_name, product_aisle))
    
    # sorting predictions highest to lowest
    ranked_products = sorted(list_of_products, key=lambda x:x[1], reverse=True)
    
    # top N reccomendations to be returned
    num_results, svd_recs = recommend_diverse_products(ranked_products, n_to_rec, aisle=rec_aisle, percent_diverse=percent_diverse)
    print('Complete')
    return num_results, svd_recs

def grocery_rater(df, num, aisle=None):
    userID = 300000
    rating_list = []
    while num > 0:
        if aisle:
            product = df[df['Aisle'].str.contains(aisle)].sample(1)
        else:
            product = df.sample(1)
        print('\n', product['Product Name'].iloc[0])
        rating = input('How do you rate this product on a scale of 1-5, choose 0 to rate a different product:\n')
        if rating == '0':
            continue
        else:
            rating_one_product = {'user_id':userID,'product_id':product['Product ID'].iloc[0],'rating':int(rating)}
            rating_list.append(rating_one_product) 
            num -= 1
    return rating_list

# return top n diverse recommendations produced
def recommend_diverse_products(ranked_products, n, aisle=None, percent_diverse=.20):
    
    num_diverse = round(n * percent_diverse)
    recs = []
    
    if n < 1:
        print('Number of recommended products must be 1 or more')
        return 0, "None"
    
    for idx, rec in enumerate(ranked_products):
        
        if n == 0:
            recommendation = pd.DataFrame(recs, columns=['Product ID', 'Rating', 'Product Name', 'Aisle'])
            return len(recs), recommendation.to_html(index=False, justify='center', classes='table1', border=2)
            
        prod_id, _, prod_name, aisle_name = [*rec]
        
               
        if aisle:                                    
            if aisle in aisle_name:                  
                if n > num_diverse:                 
                    name = prod_name
                    print('Recommendation # ', idx+1, ': ', name, '\n')
                    recs.append(rec)
                    n-= 1
                else:                                 
                    if prod_id not in short_head:     
                        name = prod_name
                        print('Recommendation # ', idx+1, ': ', name, '\n')
                        recs.append(rec)
                        n-= 1
                    else:
                        continue
            elif idx == len(ranked_products)-1:
                print('No recommended products found')
                continue
        else:
            if n > num_diverse:                  # Are we looking for a long tail product? No
                name = prod_name
                print('Recommendation # ', idx+1, ': ', name, '\n')
                recs.append(rec)
                n-= 1
            else:                                 # Are we looking for a long tail product? Yes
                if prod_id not in short_head:     # Is it NOT in the short_head list?
                    name = prod_name
                    print('Recommendation # ', idx+1, ': ', name, '\n')
                    recs.append(rec)
                    n-= 1
                else:
                    continue

def generate_new_user_recommendations(n_to_rate, n_to_rec, percent_diverse, 
                                      rate_aisle=None, rec_aisle=None):
    # Get user ratings
    user_rating = grocery_rater(products_desc, n_to_rate, aisle=rate_aisle)

    # add the new ratings to the original ratings DataFrame
    print('Creating ratings dataset...')
    new_ratings_df = new_rec_df.append(user_rating, ignore_index=True)
    new_data = Dataset.load_from_df(new_ratings_df, reader)
    
    # train a model using the new combined DataFrame
    print('Training recommendation model...')
    new_user_svd = SVD(n_factors = 20, n_epochs = 10, lr_all = 0.005, reg_all = 0.4)
    new_user_svd.fit(new_data.build_full_trainset())
    
    # make predictions for the user
    print('Making predictions...')
    list_of_products = []
    for product in new_ratings_df['product_id'].unique():
        product_name = products_desc[products_desc['Product ID'] == product]['Product Name'].iloc[0]
        product_aisle = products_desc[products_desc['Product ID'] == product]['Aisle'].iloc[0]
        list_of_products.append((product, round(new_user_svd.predict(300000, product)[3], 3), product_name, product_aisle))
    
    # order the predictions from highest to lowest rated
    ranked_products = sorted(list_of_products, key=lambda x:x[1], reverse=True)
    
    # return the top n recommendation
    num_results, svd_recs = recommend_diverse_products(ranked_products, n_to_rec, aisle=rec_aisle, percent_diverse=percent_diverse)
    return num_results, svd_recs

In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import time
from datetime import datetime
import pickle
import warnings
warnings.filterwarnings('ignore')

In [44]:
aisles = pd.read_csv("/content/aisles.csv")
departments = pd.read_csv("/content/departments.csv")
order_products_prior = pd.read_csv("/content/order_products__prior.csv")
order_products_train = pd.read_csv("/content/order_products__train.csv")
orders = pd.read_csv("/content/orders.csv")
products = pd.read_csv("/content/products.csv")

Merge order_products together
Output: merged_order_products, order_products_desc



In [45]:
merged_order_products = pd.merge(order_products_train, order_products_prior, how = 'outer')
merged_order_products.head() #merging

,order_id,product_id,add_to_cart_order,reordered
0,1,49302.0,1.0,1.0
1,1,11109.0,2.0,1.0
2,1,10246.0,3.0,0.0
3,1,49683.0,4.0,0.0
4,1,43633.0,5.0,1.0


In [46]:
products_per_order = merged_order_products.groupby('order_id').count()
products_per_order

,product_id,add_to_cart_order,reordered
order_id,,,
1,8,8,8
2,9,9,9
3,8,8,8
4,13,13,13
5,26,26,26
...,...,...,...
1341047,6,6,6
1341122,1,1,1
1341149,6,6,6


In [47]:
pro = pd.merge(products,aisles,on='aisle_id')
dep= pd.merge(pro,departments,on='department_id')
dep

,product_id,product_name,aisle_id,department_id,aisle,department
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks
1,78,Nutter Butter Cookie Bites Go-Pak,61,19,cookies cakes,snacks
2,102,Danish Butter Cookies,61,19,cookies cakes,snacks
3,172,Gluten Free All Natural Chocolate Chip Cookies,61,19,cookies cakes,snacks
4,285,Mini Nilla Wafers Munch Pack,61,19,cookies cakes,snacks
...,...,...,...,...,...,...
49683,22827,Organic Black Mission Figs,18,10,bulk dried fruits vegetables,bulk
49684,28655,Crystallized Ginger Chunks,18,10,bulk dried fruits vegetables,bulk
49685,30365,Vegetable Chips,18,10,bulk dried fruits vegetables,bulk
49686,38007,Naturally Sweet Plantain Chips,18,10,bulk dried fruits vegetables,bulk


In [48]:
order_products_desc = pd.merge(merged_order_products, dep, on = 'product_id')
order_products_desc.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department
0,1,49302.0,1.0,1.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs
1,816049,49302.0,7.0,1.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs
2,1242203,49302.0,1.0,1.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs
3,38841,49302.0,5.0,1.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs
4,45900,49302.0,19.0,0.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs


Merge user orders

In [49]:
merged_orders = pd.merge(orders, order_products_desc, on = 'order_id')
merged_orders.head() 

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department
0,1187899,1.0,train,11.0,4.0,8.0,14.0,49235.0,10.0,1.0,Organic Half & Half,53,16,cream,dairy eggs
1,1187899,1.0,train,11.0,4.0,8.0,14.0,46149.0,11.0,1.0,Zero Calorie Cola,77,7,soft drinks,beverages
2,1187899,1.0,train,11.0,4.0,8.0,14.0,196.0,1.0,1.0,Soda,77,7,soft drinks,beverages
3,1187899,1.0,train,11.0,4.0,8.0,14.0,27845.0,9.0,0.0,Organic Whole Milk,84,16,milk,dairy eggs
4,1187899,1.0,train,11.0,4.0,8.0,14.0,38928.0,3.0,1.0,0% Greek Strained Yogurt,120,16,yogurt,dairy eggs


In [50]:
test = orders[orders['eval_set']=='test']
test.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
38,2774568,3.0,test,13.0,5.0,15.0,11.0
44,329954,4.0,test,6.0,3.0,12.0,30.0
53,1528013,6.0,test,4.0,3.0,16.0,22.0
96,1376945,11.0,test,8.0,6.0,11.0,8.0
102,1356845,12.0,test,6.0,1.0,20.0,30.0


Prepare data for recommendation system

In [51]:
user_item = merged_orders[['user_id', 'product_id', 'reordered']]

In [52]:
user_item[user_item.duplicated(subset = ['user_id', 'product_id'], keep = 'first')]

,user_id,product_id,reordered
23,7.0,17638.0,1.0
24,7.0,13198.0,1.0
29,7.0,40852.0,1.0
200,66.0,8143.0,1.0
267,75.0,13176.0,1.0
...,...,...,...
96015,18474.0,42445.0,1.0
96016,18474.0,31915.0,1.0
96020,18474.0,38164.0,1.0
96093,18491.0,29668.0,1.0


In [53]:
data=[]
data_columns=['user_id','product_id','count']
for user in user_item.user_id.unique():
    counts = user_item[user_item['user_id']==user].product_id.value_counts()
    for i in range(len(counts)):
         data.append([user, counts.index[i], counts.values[i]])

In [54]:
rec_df = pd.DataFrame(data, columns=data_columns)

# Order of a user

In [55]:
namee=rec_df[rec_df['user_id']==1] #imppp
mer=pd.merge(namee , dep , on='product_id')
mer.product_name

0                Organic String Cheese
1                    Zero Calorie Cola
2     XL Pick-A-Size Paper Towel Rolls
3                Cinnamon Toast Crunch
4               Milk Chocolate Almonds
5                   Organic Whole Milk
6                           Pistachios
7           Aged White Cheddar Popcorn
8             0% Greek Strained Yogurt
9                                 Soda
10                 Organic Half & Half
Name: product_name, dtype: object

In [56]:
rec_df['count'].max()

5

In [57]:

users_150K = rec_df[rec_df['user_id'] < 150000]

In [58]:
user_info = merged_orders[['user_id', 'order_number', 'order_dow', 'order_hour_of_day', 
                           'days_since_prior_order', 'aisle']]

In [59]:
user_data = pd.get_dummies(user_info, prefix=None, columns=['aisle'])

In [60]:

user_data1 = user_data[user_data['user_id'] <= 65000]
user_data2 = user_data[(user_data['user_id'] <= 135000) & (user_data['user_id'] > 65000)]
user_data3 = user_data[user_data['user_id'] > 135000]

In [61]:
grouped_user1 = user_data1.groupby('user_id').sum()
grouped_user2 = user_data2.groupby('user_id').sum()
grouped_user3 = user_data3.groupby('user_id').sum()


In [62]:
# Go back to user_data and drop the aisle info before I group the other variables in different ways
user_data1_noaisles = user_data1.iloc[:,:5]
user_data2_noaisles = user_data2.iloc[:,:5]
user_data3_noaisles = user_data3.iloc[:,:5]

Grouping 

In [63]:
group1 = user_data1_noaisles.groupby('user_id').agg({'order_number': 'max', 'order_dow': lambda x:x.value_counts().index[0], 
                                            'order_hour_of_day': 'median', 'days_since_prior_order': 'mean'})
group2 = user_data2_noaisles.groupby('user_id').agg({'order_number': 'max', 'order_dow': lambda x:x.value_counts().index[0], 
                                            'order_hour_of_day': 'median', 'days_since_prior_order': 'mean'})
group3 = user_data3_noaisles.groupby('user_id').agg({'order_number': 'max', 'order_dow': lambda x:x.value_counts().index[0], 
                                            'order_hour_of_day': 'median', 'days_since_prior_order': 'mean'})

In [64]:
# get number of orders for each user and add to grouped_user dfs
grouped_user1['num_orders'] = group1.order_number
grouped_user2['num_orders'] = group2.order_number
grouped_user3['num_orders'] = group3.order_number

grouped_user1.drop(columns=['order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order'], inplace=True)
grouped_user2.drop(columns=['order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order'], inplace=True)
grouped_user3.drop(columns=['order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order'], inplace=True)



In [65]:
grouped_users = pd.concat([grouped_user1, grouped_user2, grouped_user3], axis=0)

In [66]:

grouped_users.head()

,aisle_air fresheners candles,aisle_asian foods,aisle_baby accessories,aisle_baby bath body care,aisle_baby food formula,aisle_bakery desserts,aisle_baking ingredients,aisle_baking supplies decor,aisle_beauty,aisle_beers coolers,aisle_body lotions soap,aisle_bread,aisle_breakfast bakery,aisle_breakfast bars pastries,aisle_bulk dried fruits vegetables,aisle_bulk grains rice dried goods,aisle_buns rolls,aisle_butter,aisle_candy chocolate,aisle_canned fruit applesauce,aisle_canned jarred vegetables,aisle_canned meals beans,aisle_canned meat seafood,aisle_cat food care,aisle_cereal,aisle_chips pretzels,aisle_cleaning products,aisle_cocoa drink mixes,aisle_coffee,aisle_cold flu allergy,aisle_condiments,aisle_cookies cakes,aisle_crackers,aisle_cream,aisle_deodorants,aisle_diapers wipes,aisle_digestion,aisle_dish detergents,aisle_dog food care,aisle_doughs gelatins bake mixes,...,aisle_packaged poultry,aisle_packaged produce,aisle_packaged seafood,aisle_packaged vegetables fruits,aisle_paper goods,aisle_pasta sauce,aisle_pickled goods olives,aisle_plates bowls cups flatware,aisle_popcorn jerky,aisle_poultry counter,aisle_prepared meals,aisle_prepared soups salads,aisle_preserved dips spreads,aisle_protein meal replacements,aisle_red wines,aisle_refrigerated,aisle_refrigerated pudding desserts,aisle_salad dressing toppings,aisle_seafood counter,aisle_shave needs,aisle_skin care,aisle_soap,aisle_soft drinks,aisle_soup broth bouillon,aisle_soy lactosefree,aisle_specialty cheeses,aisle_specialty wines champagnes,aisle_spices seasonings,aisle_spirits,aisle_spreads,aisle_tea,aisle_tofu meat alternatives,aisle_tortillas flat bread,aisle_trail mix snack mix,aisle_trash bags liners,aisle_vitamins supplements,aisle_water seltzer sparkling water,aisle_white wines,aisle_yogurt,num_orders
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,11.0
7.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,21.0
8.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,4.0
9.0,0,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,9,4.0
13.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,4.0


In [67]:
# This function came from a Medium article by Adam Ross Nelson 
def movecol(df, cols_to_move=[], ref_col='', place='After'):
    
    cols = df.columns.tolist()
    if place == 'After':
        seg1 = cols[:list(cols).index(ref_col) + 1]
        seg2 = cols_to_move
    if place == 'Before':
        seg1 = cols[:list(cols).index(ref_col)]
        seg2 = cols_to_move + [ref_col]
    
    seg1 = [i for i in seg1 if i not in seg2]
    seg3 = [i for i in cols if i not in seg1 + seg2]
    
    return(df[seg1 + seg2 + seg3])

In [68]:
grouped_users[grouped_users['num_orders']<=5]

,aisle_air fresheners candles,aisle_asian foods,aisle_baby accessories,aisle_baby bath body care,aisle_baby food formula,aisle_bakery desserts,aisle_baking ingredients,aisle_baking supplies decor,aisle_beauty,aisle_beers coolers,aisle_body lotions soap,aisle_bread,aisle_breakfast bakery,aisle_breakfast bars pastries,aisle_bulk dried fruits vegetables,aisle_bulk grains rice dried goods,aisle_buns rolls,aisle_butter,aisle_candy chocolate,aisle_canned fruit applesauce,aisle_canned jarred vegetables,aisle_canned meals beans,aisle_canned meat seafood,aisle_cat food care,aisle_cereal,aisle_chips pretzels,aisle_cleaning products,aisle_cocoa drink mixes,aisle_coffee,aisle_cold flu allergy,aisle_condiments,aisle_cookies cakes,aisle_crackers,aisle_cream,aisle_deodorants,aisle_diapers wipes,aisle_digestion,aisle_dish detergents,aisle_dog food care,aisle_doughs gelatins bake mixes,...,aisle_packaged poultry,aisle_packaged produce,aisle_packaged seafood,aisle_packaged vegetables fruits,aisle_paper goods,aisle_pasta sauce,aisle_pickled goods olives,aisle_plates bowls cups flatware,aisle_popcorn jerky,aisle_poultry counter,aisle_prepared meals,aisle_prepared soups salads,aisle_preserved dips spreads,aisle_protein meal replacements,aisle_red wines,aisle_refrigerated,aisle_refrigerated pudding desserts,aisle_salad dressing toppings,aisle_seafood counter,aisle_shave needs,aisle_skin care,aisle_soap,aisle_soft drinks,aisle_soup broth bouillon,aisle_soy lactosefree,aisle_specialty cheeses,aisle_specialty wines champagnes,aisle_spices seasonings,aisle_spirits,aisle_spreads,aisle_tea,aisle_tofu meat alternatives,aisle_tortillas flat bread,aisle_trail mix snack mix,aisle_trash bags liners,aisle_vitamins supplements,aisle_water seltzer sparkling water,aisle_white wines,aisle_yogurt,num_orders
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,4.0
9.0,0,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,9,4.0
13.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,4.0
23.0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,4,2.0
42.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18473.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,4.0
18486.0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.0
18499.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.0


In [69]:
products_with_count = rec_df.groupby('product_id').sum().drop(columns='user_id').sort_values(by='count')
products_with_count[products_with_count['count']<=5]  # 2741 products ordered 5 times or less

,count
product_id,
42437.0,1
46520.0,1
19050.0,1
35861.0,1
35862.0,1
...,...
37262.0,5
37642.0,5
37458.0,5


In [70]:
rec_df

,user_id,product_id,count
0,1.0,25133.0,1
1,1.0,46149.0,1
2,1.0,26405.0,1
3,1.0,13032.0,1
4,1.0,39657.0,1
...,...,...,...
91490,18525.0,23543.0,1
91491,18525.0,38077.0,1
91492,18525.0,9327.0,1
91493,18525.0,1308.0,1


Modeling a recommendation system

In [71]:
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import SVD
from surprise import KNNBaseline
from surprise import KNNBasic
from surprise.model_selection import train_test_split
from surprise.prediction_algorithms import knns
from surprise.similarities import cosine, msd, pearson
from surprise.prediction_algorithms import SVD, SVDpp
from surprise.model_selection import GridSearchCV
from surprise import accuracy

In [74]:
reader = Reader(rating_scale=(1, 100))

# The columns must correspond to user id, item id and ratings (in that order).
rec_data = Dataset.load_from_df(rec_df, reader)

# sample random trainset and testset
# test set is made of 25% of the ratings.
trainset, testset = train_test_split(rec_data, test_size=.25, random_state=12)

In [75]:
svd_algo = SVD()
svd_algo.fit(trainset)
svd_algo_predictions = svd_algo.test(testset)
accuracy.rmse(svd_algo_predictions)

RMSE: 0.2406


0.24060307926952187

In [76]:

accuracy.rmse(svd_algo_predictions)

RMSE: 0.2406


0.24060307926952187

In [77]:
rec_df[rec_df['user_id'] == 2555]

,user_id,product_id,count
12885,2555.0,21903.0,1
12886,2555.0,45445.0,1
12887,2555.0,33120.0,1
12888,2555.0,44632.0,1
12889,2555.0,3233.0,1
12890,2555.0,13877.0,1
12891,2555.0,22400.0,1
12892,2555.0,21938.0,1
12893,2555.0,8539.0,1
12894,2555.0,18531.0,1


In [78]:
namee=rec_df[rec_df['user_id']==2555] #imppp
mer=pd.merge(namee , dep , on='product_id')
mer.product_name

0                                  Organic Baby Spinach
1                Organic Bagged Mini Dark Peanut Butter
2                                    Organic Egg Whites
3                            Sparkling Water Grapefruit
4     Organic Turkey with Rosemary, Sage & Thyme Bon...
5                                         Smoked Salmon
6                                Peanut Butter, Classic
7                                     Green Bell Pepper
8            Organic Chicken with Lemongrass Bone Broth
9                          Organic Heavy Whipping Cream
10                             Ripe Large Pitted Olives
11                          Total Greek Strained Yogurt
12                  Uncured Hickory Smoked Sunday Bacon
13                              Baby Seedless Cucumbers
14                                 Organic Blackberries
15                                      93% Ground Beef
16                    Classic Almond Butter All Natural
17                                          Half

In [79]:
uid=196
iid=13176
# actual value was 2, prediction is 1.77
pred = svd_algo.predict(uid, iid, r_ui=2, verbose=True)

user: 196        item: 13176      r_ui = 2.00   est = 1.10   {'was_impossible': False}


In [80]:
testset[0]

(7620.0, 21709.0, 1.0)

In [81]:
pred = svd_algo.predict(127895, 6101, r_ui=2, verbose=True)

user: 127895     item: 6101       r_ui = 2.00   est = 1.03   {'was_impossible': False}


In [82]:
svd_algo_predictions[0]

Prediction(uid=7620.0, iid=21709.0, r_ui=1.0, est=1, details={'was_impossible': False})

In [83]:
param_grid = {'n_factors':[20, 100],'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs_model = GridSearchCV(SVD, param_grid=param_grid, n_jobs = -1, joblib_verbose=5)
gs_model.fit(rec_data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   32.2s
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:  3.7min finished


In [84]:
results_df = pd.DataFrame.from_dict(gs_model.cv_results)

In [85]:

svd_best_gs = SVD(n_factors = 100, n_epochs = 10, lr_all = 0.005, reg_all = 0.4)
svd_best_gs.fit(trainset)

svd_best_gs_predictions = svd_best_gs.test(testset)
# Then compute RMSE
accuracy.rmse(svd_best_gs_predictions)

# RMSE went down on grid search model from 3.4616 to 3.2483

RMSE: 0.2380


0.23804614241491173

In [86]:
max(testset,key=lambda item:item[2])
pred = svd_best_gs.predict(41356,38652, r_ui=100.0, verbose=True)
top_est = 0
top_pred = [0,0,0,0]
for uid, iid, true_r, est, _ in svd_best_gs_predictions:
    if est > top_est:
        top_est = est
        top_pred = [uid,iid,true_r,est]


user: 41356      item: 38652      r_ui = 100.00   est = 1.05   {'was_impossible': False}


In [87]:
rec_df['count'].value_counts()

1    87255
2     3839
3      338
4       57
5        6
Name: count, dtype: int64

In [88]:
rec_df['rating'] = rec_df['count'].apply(lambda x: 5 if (x >= 5) else x)

In [89]:
new_rec_df = rec_df.drop(columns='count')
new_rec_df.tail()

,user_id,product_id,rating
91490,18525.0,23543.0,1
91491,18525.0,38077.0,1
91492,18525.0,9327.0,1
91493,18525.0,1308.0,1
91494,18525.0,31717.0,1


In [90]:
rec_columns=new_rec_df.columns.values
rec_index=new_rec_df.index.values
rec_user=new_rec_df.user_id.values
rec_rating=new_rec_df.rating.values
rec_prod_id=new_rec_df.product_id.values

In [91]:
new_df = pd.DataFrame(np.column_stack([rec_user, rec_prod_id, rec_rating]), index=rec_index, columns=rec_columns)
new_df

,user_id,product_id,rating
0,1.0,25133.0,1.0
1,1.0,46149.0,1.0
2,1.0,26405.0,1.0
3,1.0,13032.0,1.0
4,1.0,39657.0,1.0
...,...,...,...
91490,18525.0,23543.0,1.0
91491,18525.0,38077.0,1.0
91492,18525.0,9327.0,1.0
91493,18525.0,1308.0,1.0


In [96]:
# Must start with a new reader

tic = time.time()
reader = Reader(rating_scale=(1, 5))
readtime = time.time()-tic
print(readtime)
# The columns must correspond to user id, item id and ratings (in that order).
toc = time.time()
new_rec_data = Dataset.load_from_df(new_rec_df, reader)
loadtime = time.time()-toc
print(loadtime)
# sample random trainset and testset
# test set is made of 25% of the ratings.
tac = time.time()
new_trainset, new_testset = train_test_split(new_rec_data, test_size=.25, random_state=12)
splittime = time.time()-tac
print(splittime)

0.00012135505676269531
0.09691953659057617
0.3065361976623535


In [97]:

new_svd_algo = SVD(n_factors = 100, n_epochs = 10, lr_all = 0.005, reg_all = 0.4)

new_svd_algo.fit(new_trainset)

new_predictions = new_svd_algo.test(new_testset)



In [98]:

param_grid = {'n_factors':[20, 100],'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
new_gs_model = GridSearchCV(SVD, param_grid=param_grid, n_jobs = -1, joblib_verbose=5)
new_gs_model.fit(new_rec_data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   31.6s
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:  3.7min finished


In [99]:
new_gs_model.best_params

{'mae': {'lr_all': 0.005, 'n_epochs': 10, 'n_factors': 20, 'reg_all': 0.6},
 'rmse': {'lr_all': 0.005, 'n_epochs': 10, 'n_factors': 20, 'reg_all': 0.4}}

In [100]:
new_results_df = pd.DataFrame.from_dict(new_gs_model.cv_results)

In [101]:
# Creating the model from the best params
new_svd_best_gs = SVD(n_factors = 20, n_epochs = 10, lr_all = 0.005, reg_all = 0.4)
new_svd_best_gs.fit(new_trainset)
new_svd_best_gs_predictions = new_svd_best_gs.test(new_testset)
# compute RMSE
accuracy.rmse(new_svd_best_gs_predictions)

RMSE: 0.2346


0.23455227816312701

In [102]:
# concat order_products_prior and train together to get a total list of ordered products
order_products_total = pd.concat([order_products_prior, order_products_train])
order_products_total = order_products_total.drop('add_to_cart_order', axis = 1)
order_products_total = order_products_total.merge(products[['product_id', 'product_name']],how='left', on='product_id')
order_products_total.head()

,order_id,product_id,reordered,product_name
0,2,33120.0,1.0,Organic Egg Whites
1,2,28985.0,1.0,Michigan Organic Kale
2,2,9327.0,0.0,Garlic Powder
3,2,45918.0,1.0,Coconut Butter
4,2,30035.0,0.0,Natural Sweetener


In [103]:

# get the list of orders that have been reordered before
reorders = order_products_total[order_products_total['reordered'] == 1]
reorders.shape

(636395, 4)

# **Collaborative functioon**

In [104]:
orders2 = orders[['order_id', 'user_id']]
user_orders = reorders.merge(orders2, on='order_id')
user_orders['high_volume'] = (user_orders['product_id'].value_counts().sort_values(ascending=False)>1)
high_volume = user_orders[user_orders['high_volume'] == True]
high_volume_users = high_volume.groupby(['user_id', 'product_name']).size().sort_values(ascending=False).unstack().fillna(0)
cosine_dists = pd.DataFrame(cosine_similarity(high_volume_users),index=high_volume_users.index, columns=high_volume_users.index)
cosine_dists.head()

user_id,7.0,27.0,42.0,46.0,55.0,56.0,68.0,71.0,75.0,79.0,86.0,90.0,96.0,110.0,116.0,118.0,119.0,126.0,137.0,138.0,144.0,146.0,147.0,150.0,153.0,159.0,164.0,187.0,197.0,208.0,210.0,212.0,214.0,223.0,226.0,229.0,233.0,234.0,236.0,240.0,...,18278.0,18286.0,18294.0,18300.0,18304.0,18308.0,18311.0,18312.0,18316.0,18332.0,18345.0,18346.0,18357.0,18373.0,18374.0,18393.0,18402.0,18409.0,18412.0,18417.0,18420.0,18424.0,18429.0,18432.0,18433.0,18440.0,18441.0,18446.0,18449.0,18462.0,18471.0,18473.0,18474.0,18488.0,18489.0,18490.0,18491.0,18493.0,18499.0,18506.0
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.57735,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
46.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
55.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [105]:
pickle.dump(high_volume, open("Pickle/highvol.p", "wb"))
pickle.dump(cosine_dists, open("Pickle/cosinedists.p", "wb"))

In [120]:
users_id=list(cosine_dists.columns)
int(users_id[0])

7

In [107]:
def Recommender_System(user_id):
    u = high_volume.groupby(['user_id','product_name']).size().sort_values(ascending=False).unstack().fillna(0)
    u_sim = pd.DataFrame(cosine_similarity(u), index=u.index, columns=u.index)

    p = high_volume.groupby(['product_name','user_id']).size().sort_values(ascending=False).unstack().fillna(0)
    
    recommendations = pd.Series(np.dot(p.values,cosine_dists[user_id]), index=p.index)
    return recommendations.sort_values(ascending=False).head(15)

In [108]:
# Get top predictions for a user

from collections import defaultdict

def get_top_n(predictions, n=10):
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
        

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    
    return top_n

In [109]:
top_n = get_top_n(new_svd_best_gs_predictions, n=20)

**Function to output recommended item by name**

In [111]:
def prediction(person):
  val=top_n[person]
  val1=[]
  for i in range (len(val)):
    val1.append(val[i][0])
  user=dep.loc[dep['product_id'].isin(val1)]
  pn=user['product_name']
  lst= list(pn)
  lst2=list(Recommender_System(person).keys())
  for item in lst2:
    if item not in lst:
      lst.append(item)
  
  
  return lst

In [112]:
pickle.dump(top_n, open("Pickle/topn.p", "wb"))

In [113]:
pickle.dump(dep, open("Pickle/dep.p", "wb"))

In [114]:
pickle.dump(rec_df, open("Pickle/recdf.p", "wb"))

In [122]:
prediction(7)

['Organic Dark Brown Sugar',
 'Raspberry Lemonade',
 'Vanilla Coffee Concentrate',
 'Seedless Red Grapes',
 'Blood Oranges',
 'Lactose Free Fat Free Milk',
 'with Crispy Almonds Cereal',
 'Grass Fed Beef Stew Meat',
 'Green Beans',
 'Greek Style Honey Yogurt',
 'Greek Nonfat Yogurt Plain',
 'Greek 100 Protein Vanilla Fat Free Yogurt',
 'Greek 0% Fat Strawberry on the Bottom Yogurt',
 'Grated Parmesan Cheese',
 'Grated Parmesan',
 'Grass milk Raw Cheddar Cheese',
 'Grapeseed Oil Vegenaise',
 'Graham Crackers',
 'Grapefruit Sparkling Water']

In [116]:
def most_ordered(person):
  namee=rec_df[rec_df['user_id']==person] #imppp
  mer=pd.merge(namee , dep , on='product_id')
  return list(mer.product_name)

In [117]:
most_ordered(90)

['Gluten Free Honey Almond Granola',
 'Organic Heritage Flakes Cereal',
 'Healthy Grains Granola Bar, Vanilla Blueberry',
 'Dark & Mint Filled Chocolate Squares',
 'Dark Chocolate Minis',
 '2% Reduced Fat Milk',
 'Sweet & Salty Nut Almond Granola Bars',
 'Peach-Pear Sparkling Water',
 'Organic Pink Lemonade Bunny Fruit Snacks',
 "Annie's Bunny Fruit Snacks Variety",
 'Fruit and Nut Bar - Peanut Butter Chocolate Chip',
 'Cherry Pie Fruit & Nut Bars',
 'Dark Chocolate Squares With Raspberry Filling',
 'Cinnamon Crunch',
 'Trail Mix Fruit & Nut Chewy Granola Bars',
 'Popped Salted Caramel Granola Bars',
 'Cool Mint Chocolate Energy Bar',
 'Chocolate Chip Energy Bars',
 'Dark & Sea Salt Caramel Chocolate Squares',
 'Maple Pumpkin Seeds with Sea Salt Chewy with a Crunch Granola Bars',
 'Twilight Delight 72% Cacao Intense Dark Squares',
 'Annie Peely Fruit 4.5 Z',
 'Sea Salt Soiree Intense Dark Chocolate Squares',
 'Organic Graham Crunch Cereal',
 'Squares Milk & Caramel Chocolate',
 'Sparkl